# RDF Conversions

This notebook demonstrates how to [convert](https://nexus-forge.readthedocs.io/en/latest/interaction.html#converting) data in any RDF format (n3, Turtle, ...) to Resources and vice-versa. 

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource

## Resource to rdflib.Graph

In [4]:
jane = Resource(type="Person", name="Jane Doe")

In [5]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [6]:
graph = forge.as_graph(jane)

In [7]:
len(graph) == 2

True

In [8]:
for triple in graph:
    print(triple)

(rdflib.term.URIRef('https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/22ec9959-ab76-419a-9864-131b0dc12d57'), rdflib.term.URIRef('http://schema.org/name'), rdflib.term.Literal('Jane Doe'))
(rdflib.term.URIRef('https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/22ec9959-ab76-419a-9864-131b0dc12d57'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://schema.org/Person'))


In [9]:
graph_store_metadata = forge.as_graph(jane, store_metadata=True)
len(graph_store_metadata)

14

In [10]:
for triple in graph_store_metadata:
    print(triple)

(rdflib.term.URIRef('https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/22ec9959-ab76-419a-9864-131b0dc12d57'), rdflib.term.URIRef('http://schema.org/name'), rdflib.term.Literal('Jane Doe'))
(rdflib.term.URIRef('https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/22ec9959-ab76-419a-9864-131b0dc12d57'), rdflib.term.URIRef('https://bluebrain.github.io/nexus/vocabulary/updatedAt'), rdflib.term.Literal('2022-04-12T22:47:28.984000+00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))
(rdflib.term.URIRef('https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/22ec9959-ab76-419a-9864-131b0dc12d57'), rdflib.term.URIRef('https://bluebrain.github.io/nexus/vocabulary/rev'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/22ec9959-ab76-419a-9864-131b0dc12d57'), rdflib.term.URIRef('https://bluebrain.github.io/nexus/vocabulary/createdAt'), rdflib.

## rdflib.Graph to Resource

In [11]:
#data in n3 format. Other RDF formats can be used
data = '''
@prefix schema: <http://schema.org/> .
[] a schema:Person;
     schema:name "Jane Doe" ;
     schema:knows <http://example.org/JohnDoe>;
     schema:affiliation <https://www.grid.ac/institutes/grid.5333.6>.

<http://example.org/JohnDoe> a schema:Person;
     schema:name "John Doe" .
     
<https://www.grid.ac/institutes/grid.5333.6> a schema:Organization;
     schema:name "EPFL" .
'''

In [12]:
import rdflib
graph = rdflib.Graph()
graph.parse(data=data, format="n3")

<Graph identifier=Nabbd6999596d410b9d9760651f8b8bcd (<class 'rdflib.graph.Graph'>)>

In [13]:
len(graph)==8

True

In [14]:
resources = forge.from_graph(graph)

In [15]:
len(resources) == 3

True

### Apply a JSON-LD Frame

In [16]:
frame = {
        "@type": [ 'http://schema.org/Person'],
        "@embed": True
}  
resources = forge.from_graph(data=graph, frame= frame)

In [17]:
len(resources) == 2

True

In [18]:
forge.as_jsonld(resources)

[{'@context': {'brick': 'https://brickschema.org/schema/Brick#',
   'csvw': 'http://www.w3.org/ns/csvw#',
   'dc': 'http://purl.org/dc/elements/1.1/',
   'dcam': 'http://purl.org/dc/dcam/',
   'dcat': 'http://www.w3.org/ns/dcat#',
   'dcmitype': 'http://purl.org/dc/dcmitype/',
   'dcterms': 'http://purl.org/dc/terms/',
   'doap': 'http://usefulinc.com/ns/doap#',
   'foaf': 'http://xmlns.com/foaf/0.1/',
   'odrl': 'http://www.w3.org/ns/odrl/2/',
   'org': 'http://www.w3.org/ns/org#',
   'owl': 'http://www.w3.org/2002/07/owl#',
   'prof': 'http://www.w3.org/ns/dx/prof/',
   'prov': 'http://www.w3.org/ns/prov#',
   'qb': 'http://purl.org/linked-data/cube#',
   'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
   'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
   'schema': 'https://schema.org/',
   'schema1': 'http://schema.org/',
   'sh': 'http://www.w3.org/ns/shacl#',
   'skos': 'http://www.w3.org/2004/02/skos/core#',
   'sosa': 'http://www.w3.org/ns/sosa/',
   'ssn': 'http://www.w3

### Select a Type

In [19]:
_type= ["http://schema.org/Organization"]
resources = forge.from_graph(data=graph, type= _type,use_model_context=True)

In [20]:
isinstance(resources, Resource)

True

In [ ]:
print(resources)

In [22]:
isinstance(resources, Resource)

True

In [23]:
epfl_json = {'id': 'https://www.grid.ac/institutes/grid.5333.6',
  'type': 'Organization',
  'name': 'EPFL'}

In [24]:
epfl_json == forge.as_json(resources)

True

### Use the forge Model context

In [25]:
_type= ["http://schema.org/Organization"]
resources = forge.from_graph(data=graph, type= _type, use_model_context=True)

In [26]:
isinstance(resources, Resource)

True